# G201

## Set up

In [1]:
from fastcore.test import *
from fastcore.utils import *
import pandas as pd
from pandas import DataFrame
import numpy as np
import pyspssio

import raine_tools as rn
from data_harmonising.data import *
from data_harmonising.transforms import *
from data_harmonising.tests import *

from src.config import WORKBOOK, SHEETNAME

In [2]:
INPUT = "../data/raw/G201_Quest_PA.sav"
OUTPUT = "../data/interim/G201_Quest_PA.sav"
FINAL = "../data/processed/G201_Quest_PA.sav"

In [3]:
df, meta = read_sav(INPUT, index="ID")

## Data exploration

### The strange case of `G201_IF13`

Why does `G201_IF13` exist, especially when there's no corresponding `G201_IC13`?  
Furthermore, the values don't align with the previous IC? variables.

In [4]:
df.loc[:, ["G201_IF13"]].value_counts() 

G201_IF13
0.0          2414
9.0            27
1.0             4
2.0             1
Name: count, dtype: int64

For all other cases of `G201_IF1` to `G201_IF12`, a 0 means that the incident is related to the incident in the previous IF? variable.  
When `G201_IF13` is 0, the majority of cases were N/A, and values range across the board; this doesn't fit with the existing logic.

In [5]:
df.loc[df["G201_IF13"] == 0, "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
88.0       0.0          2404
1.0        0.0             6
0.0        0.0             2
2.0        0.0             1
999.0      0.0             1
Name: count, dtype: int64

At least when `G201_IF13` is missing (9), the previous value was also missing (999).

In [6]:
df.loc[df["G201_IF13"] == 9, "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
999.0      9.0          27
Name: count, dtype: int64

Finally, when `G201_IF13` is 1 or 2, the previous values are 0 or 1; this doesn't specifically highlight any errors.

In [7]:
df.loc[df["G201_IF13"].between(1, 2), "G201_IF12":"G201_IF13"].value_counts()

G201_IF12  G201_IF13
1.0        1.0          3
0.0        1.0          1
1.0        2.0          1
Name: count, dtype: int64

`G201_IF13` does not align with the previous values, and there are clear logical discrepancies.  
Given there is no corresponding `G201_IC13` to explain the few instances of a 1 or 2, and the 0s appear to be unreliable, I plan to drop it.

### Check if there are variables that don't contain data

Verify that all columns contain at least one row with data, and drop any that don't.

`G201_IF12` and `G201_IC12` contain values.

In [8]:
df["G201_IF12"].value_counts()

G201_IF12
88.0     2404
999.0      28
1.0        10
0.0         3
2.0         1
Name: count, dtype: int64

`G201_INF5` and `G201_INC5` don't contain data: **drop them**.

In [9]:
df["G201_INF5"].value_counts()

G201_INF5
0.0      2423
999.0      23
Name: count, dtype: int64

`G201_HOH6`, `G201_HOC6`, and `G201_HOF6` contain values.

In [10]:
df["G201_HOH6"].value_counts()

G201_HOH6
0.0      2416
999.0      22
103.0       6
631.0       1
104.0       1
Name: count, dtype: int64

### Checking that IF? and IC? variables are correctly aligned

In [11]:
IF_ROWS = ['G201_IF1', 'G201_IF2', 'G201_IF3', 'G201_IF4', 'G201_IF5', 'G201_IF6',
           'G201_IF7', 'G201_IF8', 'G201_IF9', 'G201_IF10', 'G201_IF11', 'G201_IF12']

IC_ROWS = ['G201_IC1', 'G201_IC2', 'G201_IC3', 'G201_IC4', 'G201_IC5', 'G201_IC6',
           'G201_IC7', 'G201_IC8', 'G201_IC9', 'G201_IC10', 'G201_IC11', 'G201_IC12']

If `G201_IC?` is N/A ("88888"), the corresponding `G201_IF?` should also be N/A (88).  
Else if `G201_IC?` is Missing ("99999"), the corresponding `G201_IF?` should also be Missing (999).    
Otherwise, the corresponding `G201_IF?` should not be N/A or Missing.

In [12]:
for n in range(1, 13): 
    assert (df.loc[df[f"G201_IC{n}"] == "88888", f"G201_IF{n}"] == 88).all()
    assert (df.loc[df[f"G201_IC{n}"] == "99999", f"G201_IF{n}"] == 999).all()
    assert not df.loc[~df[f"G201_IC{n}"].isin(["88888", "99999"]), f"G201_IF{n}"].isin([88, 999]).all()

The reverse of this should also be true, such that if `G201_IF?` is N/A, the corresponding `G201_IC?` value should also be N/A, and so on.

In [13]:
for n in range(1, 13): 
    assert (df.loc[df[f"G201_IF{n}"] == 88, f"G201_IC{n}"] == "88888").all()
    assert (df.loc[df[f"G201_IF{n}"] == 999, f"G201_IC{n}"] == "99999").all()
    assert not df.loc[~df[f"G201_IF{n}"].isin([88, 999]), f"G201_IC{n}"].isin(["88888", "99999"]).all()

If `G201_IF1` is No (0), then remaining `G201_IF?` values should be N/A (88).  
However, a single instance of this was located at ID 10080. 

In [14]:
df.loc[df["G201_IF1"] == 0].filter(regex=r'I(C|F)[1-5]\b')

,G201_IC1,G201_IC2,G201_IC3,G201_IC4,G201_IC5,G201_IF1,G201_IF2,G201_IF3,G201_IF4,G201_IF5
ID,,,,,,,,,,
10080.0,69100,69290,88888,88888,88888,0.0,1.0,88.0,88.0,88.0


For ID 10080, given both conditions are interrelatated (dermatitis), I believe it's likely the participant would have seen the doctor once for both cases.  
Therefore, it makes sense to change IF1 to 1 and IF2 to 0.

In [15]:
idx = [10080]
df.loc[idx, "G201_IF1"] = 1
df.loc[idx, "G201_IF2"] = 0

Participants with IDs 17890, 19270, and 20370 had missing values in place of N/A values, detected during testing process.

In [16]:
idx = [17890, 19270, 20370]
df.loc[idx, "G201_IC1":"G201_IF12"] = df.loc[idx, "G201_IC1":"G201_IF12"].replace({"99999": "88888", 999: 88})

In [17]:
# If any IF or IC value is missing, all others in that row are also missing
for index, row in df.iterrows():
    if row[IF_ROWS].isin([999]).any():
        assert row[IF_ROWS].isin([999]).all(), f"ID {index}: Not all IF values are missing when one is missing."
    if row[IC_ROWS].isin(['99999']).any():
        assert row[IC_ROWS].isin(['99999']).all(), f"ID {index}: Not all IC values are missing when one is missing."

ID 11642: two values for IF and IC that follow a N/A value.
It appears a couple of cells of data were accidentally copied down a line, for IF3, IF4, IC3 and IC4.

ID 23180: one value for IF and IC that follows a N/A value.
Given IF4 and IC4 match IF2 and IC2, and there's a N/A value in-between, it seems likely they've been accidentally copied over.

In [18]:
df.loc[11642, ["G201_IF3", "G201_IF4"]] = 88
df.loc[11642, ["G201_IC3", "G201_IC4"]] = "88888"

df.loc[23180, ["G201_IF4"]] = 88
df.loc[23180, ["G201_IC4"]] = "88888"

In [19]:
# If any IF or IC value is N/A, all following values are also N/A
for index, row in df.iterrows():
    if row[IF_ROWS].isin([88]).any():
        assert row[IF_ROWS].loc[row[IF_ROWS].isin([88]).idxmax():].isin([88]).all(), f"Row {index}: Not all following IF values are N/A."
    if row[IC_ROWS].isin(['88888']).any():
        assert row[IC_ROWS].loc[row[IC_ROWS].isin(['88888']).idxmax():].isin(['88888']).all(), f"Row {index}: Not all following IC values are N/A."

Finally, confirm that if the first variable for all of `G201_IF1`, `G201_INF1`, and `G201_HOF1` are 88, all subsequent values are 88.

In [20]:
assert all(df.loc[df["G201_IF1"] == 88, "G201_IF1":"G201_IF12"] == 88)
assert all(df.loc[df["G201_INF1"] == 88, "G201_INF1":"G201_INF5"] == 88)
assert all(df.loc[df["G201_HOF1"] == 88, "G201_HOF1":"G201_HOF6"] == 88)

### Check ICD9 codes

Verify all `INC1` codes are either E- or V-codes.

In [21]:
dfx = df.loc[~df["G201_INC1"].isin(["88888", "99999"]), "G201_INC1"].copy()
assert np.all(dfx.str.startswith(("E", "V")))

Verify all cases where no frequency for illness was reported is aligned with no ICD-9 code for that illness being reported.  
Do the same for injuries, and hospital admissions.

In [22]:
test_eq(np.where(df["G201_IF1"] == 88), np.where(df["G201_IC1"] == "88888"))
# test_eq(np.where(df["G201_INF1"] == 0), np.where(df["G201_INC1"] == "88888"))
# test_eq(np.where(df["G201_HOF1"] == 0), np.where(df["G201_HOC1"] == "88888"))

### Hospital Admission Variables

The problem is extremely pronounced for hospital admissions.
There is a huge range of cases where there is misalignment; the frequency is 0 when there is a valid ICD-9 code and hospital code, and in other instances, there is a frequency of 1 or more when there is no valid ICD-9 code and/or hospital code.
Comparing back to a previous, superseded version of the dataset from 2013 shows this data hasn't been changed in that time.
Therefore, this is either reflective of the data that was captured, or errors introduced earlier than 2013, meaning they can't be verified.

It appears, from investigating the data, that even though it's not been labelled as such, 0s are in some cases used to indicate that there WAS a hospital admission, and it was for multiple ICD-9 codes. 
I've subsequently tried to identify where 0s simply represented "N/A" and have recoded those to 88 (when the corresponding ICD-9 code and hospital code were both not provided).

In [23]:
vars = [
    ("G201_HOF1", "G201_HOC1", "G201_HOH1"),
    ("G201_HOF2", "G201_HOC2", "G201_HOH2"),
    ("G201_HOF3", "G201_HOC3", "G201_HOH3"),
    ("G201_HOF4", "G201_HOC4", "G201_HOH4"),
    ("G201_HOF5", "G201_HOC5", "G201_HOH5"),
    ("G201_HOF6", "G201_HOC6", "G201_HOH6"),
]

for a, b, c in vars:
    m1 = (df[a] == 0) & (df[b] == "88888") & (df[c] == 0)
    m2 = (df[a] == 0) | (df[b] == "88888") | (df[c] == 0)
    print(df.loc[~m1 & m2, [a, b, c]])
    print("")

         G201_HOF1 G201_HOC1  G201_HOH1
ID                                     
10060.0        1.0     46610        0.0
10180.0        1.0     03390        0.0
10210.0        1.0     S4572        0.0
10290.0        1.0     S2161        0.0
10510.0        1.0     78060        0.0
...            ...       ...        ...
54350.0        1.0     88888        0.0
54360.0        1.0     88888        0.0
54400.0        1.0     88888        0.0
54530.0        1.0     88888        0.0
54540.0        1.0     88888        0.0

[164 rows x 3 columns]

         G201_HOF2 G201_HOC2  G201_HOH2
ID                                     
10290.0        0.0     S2830        0.0
10610.0        1.0     S2759        0.0
10840.0        1.0     51980        0.0
11010.0        1.0     78030        0.0
11030.0        1.0     78030        0.0
...            ...       ...        ...
52790.0        1.0     S0943        0.0
53380.0        1.0     88888        0.0
53630.0        1.0     88888        0.0
54062.0        

In [24]:
# For example, a subset of IDs with the aforementioned problems
df.loc[[40690, 46330, 54590], ["G201_HOF6", "G201_HOC6", "G201_HOH6"]]

,G201_HOF6,G201_HOC6,G201_HOH6
ID,,,
40690.0,0.0,27130,103.0
46330.0,0.0,55890,103.0
54590.0,0.0,37600,103.0


There's a singular example where HOF1 has the value No (0), where there is a corresponding ICD-9 code and hospital code; change the value for HOF1 to Missing (999).

In [25]:
df.loc[(df["G201_HOF1"] == 0) & ((df["G201_HOC1"] != "88888") | (df["G201_HOH1"] != 0)), ["G201_HOF1", "G201_HOC1", "G201_HOH1"]]

,G201_HOF1,G201_HOC1,G201_HOH1
ID,,,
42320.0,0.0,S5310,985.0


In [26]:
df.at[42320, "G201_HOF1"] = 999

The following code identifies where all frequency values (from the second variable onwards) have been recorded as 0, and there is no corresponding ICD-9 code.
It then changes those values from 0 to 88. For HOF1, all values of 0 should be classified as N/A, as there are no "previous visit" to refer to.

In [27]:
idx_HOF1 = df.loc[(df["G201_HOF1"] == 0)].index
idx_HOF2 = df.loc[(df["G201_HOF2"] == 0) & (df["G201_HOC2"] == "88888")].index
idx_HOF3 = df.loc[(df["G201_HOF3"] == 0) & (df["G201_HOC3"] == "88888")].index
idx_HOF4 = df.loc[(df["G201_HOF4"] == 0) & (df["G201_HOC4"] == "88888")].index
idx_HOF5 = df.loc[(df["G201_HOF5"] == 0) & (df["G201_HOC5"] == "88888")].index
idx_HOF6 = df.loc[(df["G201_HOF6"] == 0) & (df["G201_HOC6"] == "88888")].index

idxs = [
    (idx_HOF1, "G201_HOF1"),
    (idx_HOF2, "G201_HOF2"), 
    (idx_HOF3, "G201_HOF3"), 
    (idx_HOF4, "G201_HOF4"), 
    (idx_HOF5, "G201_HOF5"), 
    (idx_HOF6, "G201_HOF6")
]

for idx, var in idxs:
    df.loc[idx, var] = 88

Recode the hospital codes also, such that if HOH is 0, and there is no valid ICD-9 code, HOH = -88 (N/A), and if there **is** a valid ICD-9 code, HOH = -99 (Missing).

In [28]:
idx_HOH1_NA = df.loc[(df["G201_HOH1"] == 0) & (df["G201_HOC1"] == "88888"), ["G201_HOH1"]].index
idx_HOH1_MISS = df.loc[(df["G201_HOH1"] == 0) & (df["G201_HOC1"] != "88888"), ["G201_HOH1"]].index
idx_HOH2_NA = df.loc[(df["G201_HOH2"] == 0) & (df["G201_HOC2"] == "88888"), ["G201_HOH2"]].index
idx_HOH2_MISS = df.loc[(df["G201_HOH2"] == 0) & (df["G201_HOC2"] != "88888"), ["G201_HOH2"]].index
idx_HOH3_NA = df.loc[(df["G201_HOH3"] == 0) & (df["G201_HOC3"] == "88888"), ["G201_HOH3"]].index
idx_HOH3_MISS = df.loc[(df["G201_HOH3"] == 0) & (df["G201_HOC3"] != "88888"), ["G201_HOH3"]].index
idx_HOH4_NA = df.loc[(df["G201_HOH4"] == 0) & (df["G201_HOC4"] == "88888"), ["G201_HOH4"]].index
idx_HOH4_MISS = df.loc[(df["G201_HOH4"] == 0) & (df["G201_HOC4"] != "88888"), ["G201_HOH4"]].index
idx_HOH5_NA = df.loc[(df["G201_HOH5"] == 0) & (df["G201_HOC5"] == "88888"), ["G201_HOH5"]].index
idx_HOH5_MISS = df.loc[(df["G201_HOH5"] == 0) & (df["G201_HOC5"] != "88888"), ["G201_HOH5"]].index
idx_HOH6_NA = df.loc[(df["G201_HOH6"] == 0) & (df["G201_HOC6"] == "88888"), ["G201_HOH6"]].index
idx_HOH6_MISS = df.loc[(df["G201_HOH6"] == 0) & (df["G201_HOC6"] != "88888"), ["G201_HOH6"]].index

idxs = [
    (idx_HOH1_NA, idx_HOH1_MISS, "G201_HOH1"),
    (idx_HOH2_NA, idx_HOH2_MISS, "G201_HOH2"),
    (idx_HOH3_NA, idx_HOH3_MISS, "G201_HOH3"),
    (idx_HOH4_NA, idx_HOH4_MISS, "G201_HOH4"),
    (idx_HOH5_NA, idx_HOH5_MISS, "G201_HOH5"),
    (idx_HOH6_NA, idx_HOH6_MISS, "G201_HOH6"),
]

for na, miss, var in idxs:
    df.loc[na, var] = 88
    df.loc[miss, var] = 999

### Injury variables

There are 29 cases where `G201_IC1` is N/A where `G201_IF1` is not 0; the variables are misaligned.
This could be because the IF1 value is incorrect, there have been errors introduced into the data, or perhaps missing data was incorrectly labelled as N/A.
Without further information available, this data has been left as is, and is up to researchers to determine how to respond.

In [29]:
vars = [
    ("G201_INF1", "G201_INC1"),
    ("G201_INF2", "G201_INC2"),
    ("G201_INF3", "G201_INC3"),
    ("G201_INF4", "G201_INC4"),
    ("G201_INF5", "G201_INC5"),
]

for a, b in vars:
    m1 = (df[a] == 0) & (df[b] == "88888")
    m2 = (df[a] == 0) | (df[b] == "88888")
    print(df.loc[~m1 & m2, [a, b]])
    print("")

         G201_INF1 G201_INC1
ID                          
21950.0        4.0     88888
23570.0        1.0     88888
51750.0        1.0     88888
52020.0        1.0     88888
52031.0        1.0     88888
52060.0        2.0     88888
52080.0        1.0     88888
52120.0        1.0     88888
52140.0        1.0     88888
52170.0        1.0     88888
53200.0        1.0     88888
53340.0        1.0     88888
53360.0        1.0     88888
53450.0        1.0     88888
53460.0        1.0     88888
53490.0        1.0     88888
53500.0        1.0     88888
53560.0        1.0     88888
53590.0        1.0     88888
53600.0        1.0     88888
53720.0        1.0     88888
53790.0        1.0     88888
53880.0        1.0     88888
53890.0        1.0     88888
54230.0        1.0     88888
54250.0        1.0     88888
54350.0        2.0     88888
54400.0        1.0     88888
54470.0        1.0     88888

         G201_INF2 G201_INC2
ID                          
45380.0        0.0     10000
51750.0      

0s only represent "N/A"; convert ALL 0s to 88s.

In [30]:
idx_INF2 = df.loc[df["G201_INF2"] == 0].index
idx_INF1 = df.loc[df["G201_INF1"] == 0].index
idx_INF3 = df.loc[df["G201_INF3"] == 0].index
idx_INF4 = df.loc[df["G201_INF4"] == 0].index

idxs = [
    (idx_INF1, "G201_INF1"),
    (idx_INF2, "G201_INF2"),
    (idx_INF3, "G201_INF3"),
    (idx_INF4, "G201_INF4"),
]

for idx, var in idxs:
    df.loc[idx, var] = 88

## Create, Rename, and Delete Variables

### Reading changes from the spreadsheet

In [31]:
wb = pd.read_excel(WORKBOOK, sheet_name=SHEETNAME)
wb = wb[wb["Dataset"] == "G201"]
wb['vn_old'] = wb['Dataset'] + '_' + wb['Variable Name']
wb['vn_new'] = wb['Dataset'] + '_' + wb['Harmonised Variable Name']

In [32]:
var_map = wb[["vn_old", "vn_new"]].dropna()
var_map = var_map.set_index("vn_old").to_dict()["vn_new"]

### Create

Create following binary variables (for all cases: in past 12 months that required a visit to hospital)
- `G201_ILL`: illnesses or problems (Yes if `G201_IF1` is Yes); 
- `G201_INJ`: any accidents or injuries (Yes if `G201_INF1` is Yes); and
- `G201_HOSP`: admitted to hospital (Yes if `G201_HO1` is Yes)

In [33]:
conditions = [
    # When the value for the conditional col is Missing (999) set to Missing
    {"condition": 999, "value": -99},
    # When value is N/A (88), set to 0 (No)
    {"condition": 88, "value": 0},
    # When value is No (0), retain the 0
    {"condition": 0, "value": 0},
    # When value is neither No, N/A nor Missing, set to 1 (Yes)
    {"condition": lambda x: x not in [0, 88, 999], "value": 1}
]

CREATE = [
    {
        "target_col": "G201_ILL",
        "condition_col": "G201_IF1",
        "conditions": conditions
    },
    {
        "target_col": "G201_INJ",
        "condition_col": "G201_INF1",
        "conditions": conditions
    },
    {
        "target_col": "G201_HOSP",
        "condition_col": "G201_HOF1",
        "conditions": conditions
    }
]

In [34]:
df, meta = create_variables(df, meta, CREATE)

In [35]:
# Shift added columns to the same section of the dataset
df = reorder_columns(df, ["G201_ILL",  "G201_INJ", "G201_HOSP"], after_column="G201_IMPLC")

Test to ensure changes were correctly implemented.

In [36]:
TEST = {
    "G201_ILL": "G201_IF1", 
    "G201_INJ": "G201_INF1",
    "G201_HOSP": "G201_HOF1"
}

for target_col, condition_col in TEST.items():
    assert all(df.loc[~df[condition_col].isin([0, 88, 999]), target_col] == 1) 
    assert all(df.loc[df[condition_col] == 0, target_col] == 0)
    assert all(df.loc[df[condition_col] == 88, target_col] == 0) 
    assert all(df.loc[df[condition_col] == 999, target_col] == -99) 

### Delete

In [37]:
DELETE = [
    "DOB", # should not be included in the dataset; captured in Longitudinal Data/Gen2_Ages.sav
    "G201_IF13",
    "G201_INF5",
    "G201_INC5"
]

In [38]:
df, meta = delete_variables(df, meta, DELETE)

### Rename

In [39]:
RENAME = {
    'G201_IF1': 'G201_ILL_N_1',
    'G201_IF2': 'G201_ILL_N_2',
    'G201_IF3': 'G201_ILL_N_3',
    'G201_IF4': 'G201_ILL_N_4',
    'G201_IF5': 'G201_ILL_N_5',
    'G201_IF6': 'G201_ILL_N_6',
    'G201_IF7': 'G201_ILL_N_7',
    'G201_IF8': 'G201_ILL_N_8',
    'G201_IF9': 'G201_ILL_N_9',
    'G201_IF10': 'G201_ILL_N_10',
    'G201_IF11': 'G201_ILL_N_11',
    'G201_IF12': 'G201_ILL_N_12',
    'G201_IC1': 'G201_ILL_ICD9_1',
    'G201_IC2': 'G201_ILL_ICD9_2',
    'G201_IC3': 'G201_ILL_ICD9_3',
    'G201_IC4': 'G201_ILL_ICD9_4',
    'G201_IC5': 'G201_ILL_ICD9_5',
    'G201_IC6': 'G201_ILL_ICD9_6',
    'G201_IC7': 'G201_ILL_ICD9_7',
    'G201_IC8': 'G201_ILL_ICD9_8',
    'G201_IC9': 'G201_ILL_ICD9_9',
    'G201_IC10': 'G201_ILL_ICD9_10',
    'G201_IC11': 'G201_ILL_ICD9_11',
    'G201_IC12': 'G201_ILL_ICD9_12',
    'G201_INF1': 'G201_INJ_N_1',
    'G201_INF2': 'G201_INJ_N_2',
    'G201_INF3': 'G201_INJ_N_3',
    'G201_INF4': 'G201_INJ_N_4',
    'G201_INC1': 'G201_INJ_ICD9_1',
    'G201_INC2': 'G201_INJ_ICD9_2',
    'G201_INC3': 'G201_INJ_ICD9_3',
    'G201_INC4': 'G201_INJ_ICD9_4',
    'G201_HOH1': 'G201_HOSP_CODE_1',
    'G201_HOH2': 'G201_HOSP_CODE_2',
    'G201_HOH3': 'G201_HOSP_CODE_3',
    'G201_HOH4': 'G201_HOSP_CODE_4',
    'G201_HOH5': 'G201_HOSP_CODE_5',
    'G201_HOH6': 'G201_HOSP_CODE_6',
    'G201_HOC1': 'G201_HOSP_ICD9_1',
    'G201_HOC2': 'G201_HOSP_ICD9_2',
    'G201_HOC3': 'G201_HOSP_ICD9_3',
    'G201_HOC4': 'G201_HOSP_ICD9_4',
    'G201_HOC5': 'G201_HOSP_ICD9_5',
    'G201_HOC6': 'G201_HOSP_ICD9_6',
    'G201_HOF1': 'G201_HOSP_N_1',
    'G201_HOF2': 'G201_HOSP_N_2',
    'G201_HOF3': 'G201_HOSP_N_3',
    'G201_HOF4': 'G201_HOSP_N_4',
    'G201_HOF5': 'G201_HOSP_N_5',
    'G201_HOF6': 'G201_HOSP_N_6'
}

In [40]:
df, meta = rename_variables(df, meta, RENAME)

## Harmonise Metadata

In [41]:
cols = list(var_map.values())
meta.loc[["Label", "Field Values"], cols]

,G201_ILL_N_1,G201_ILL_N_2,G201_ILL_N_3,G201_ILL_N_4,G201_ILL_N_5,G201_ILL_N_6,G201_ILL_N_7,G201_ILL_N_8,G201_ILL_N_9,G201_ILL_N_10,...,G201_HOSP_ICD9_3,G201_HOSP_ICD9_4,G201_HOSP_ICD9_5,G201_HOSP_ICD9_6,G201_HOSP_N_1,G201_HOSP_N_2,G201_HOSP_N_3,G201_HOSP_N_4,G201_HOSP_N_5,G201_HOSP_N_6
Label,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,Illness no.times,...,Hospital ICD-9 code,Hospital ICD-9 code,Hospital ICD-9 code,Hospital ICD-9 code,Hospital Number of times,Hospital Number of times,Hospital Number of times,Hospital Number of times,Hospital Number of times,Hospital Number of times
Field Values,{999.0: 'missing'},"{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...","{0.0: 'Related to previous illnesses', 88.0: '...",...,{'99999': 'missing'},{'99999': 'missing'},{'99999': 'missing'},{'99999': 'missing'},{999.0: 'missing'},{999.0: 'missing'},{999.0: 'missing'},{999.0: 'missing'},{999.0: 'missing'},{999.0: 'missing'}


Investigate 0s for ILL_N_? INJ_N_? and HOSP_N_?

Verify a value of 0 for `ILL_N_2` is not just a missing or N/A value, and contains legitimate data

In [42]:
assert ~np.any(df.loc[df['G201_ILL_N_2'] == 0, ['G201_ILL_ICD9_2']] == "88888")
assert np.all(df.loc[df['G201_ILL_N_2'] == 88, ['G201_ILL_ICD9_2']] == "88888")

FOR `INJ_N`, 0 appears to refer to N/A in all cases.

In [43]:
df['G201_INJ_N_1'].value_counts()

G201_INJ_N_1
88.0     2045
1.0       364
999.0      23
2.0        10
4.0         3
3.0         1
Name: count, dtype: int64

When `INJ_N_1` != 0 and `INJ_N_2` == 0, the following variables also have a value of 0.

In [44]:
df.loc[(df['G201_INJ_N_2'] == 0) & (df['G201_INJ_N_1'] != 0), ['G201_INJ_N_1', 'G201_INJ_N_2', 'G201_INJ_N_3', 'G201_INJ_N_4']].value_counts()

Series([], Name: count, dtype: int64)

When `INJ_N_1` != 0 and `INJ_N_2` == 0, all corresponding values for `INJ_ICD9` are "88888" (N/A).

In [45]:
df.loc[(df['G201_INJ_N_2'] == 0) & (df['G201_INJ_N_1'] != 0), ['G201_INJ_ICD9_2']].value_counts()

Series([], Name: count, dtype: int64)

For `HOSP_N`, 

In [46]:
df['G201_HOSP_N_1'].value_counts()

G201_HOSP_N_1
88.0     1957
1.0       445
999.0      23
2.0        16
3.0         3
4.0         1
5.0         1
Name: count, dtype: int64

When HOSP_N_1 == 0, HOSP_ICD9_1 and HOSP_CODE_1 should be Missing or N/A.  
However, in one case (ID: 42320), there is a real value for HOSP_ICD9_1 and HOSP_CODE_1.  
TODO: Investigate if there was previously a value. If not, recode HOSP_N_1 to Missing.

In [47]:
# assert np.all(df.loc[df['G201_HOSP_N_1'] == 0, ["G201_HOSP_ICD9_1"]] == "88888")
# assert np.all(df.loc[df['G201_HOSP_N_1'] == 0, ["G201_HOSP_CODE_1"]] == 0)
df.loc[(df['G201_HOSP_N_1'] == 0) & (df["G201_HOSP_ICD9_1"] != "88888"), ["G201_HOSP_ICD9_1"]]
df.loc[(df['G201_HOSP_N_1'] == 0) & (df["G201_HOSP_CODE_1"] != 0), ["G201_HOSP_CODE_1"]]

,G201_HOSP_CODE_1
ID,


When HOSP_N_1 == 0, all subsequent values for HOSP_N_?, HOSP_CODE_?, and HOSP_ICD9_? should be N/A.  
TODO: Change these values for HOSP_N and HOSP_CODE to appropriate N/A values.

In [48]:
HOSP_N = ['G201_HOSP_N_2', 'G201_HOSP_N_3', 'G201_HOSP_N_4', 'G201_HOSP_N_5', 'G201_HOSP_N_6']
HOSP_CODE = ['G201_HOSP_CODE_2', 'G201_HOSP_CODE_3', 'G201_HOSP_CODE_4', 'G201_HOSP_CODE_5', 'G201_HOSP_CODE_6']
HOSP_ICD9 = ['G201_HOSP_ICD9_2', 'G201_HOSP_ICD9_3', 'G201_HOSP_ICD9_4', 'G201_HOSP_ICD9_5', 'G201_HOSP_ICD9_6']

assert np.all(df.loc[(df['G201_HOSP_N_1'] == 0), HOSP_N] == 88)
assert np.all(df.loc[(df['G201_HOSP_N_1'] == 0), HOSP_CODE] == 0)
assert np.all(df.loc[(df['G201_HOSP_N_1'] == 0), HOSP_ICD9] == "88888")

In [49]:
df.loc[(df['G201_HOSP_N_2'] == 0) & (df['G201_HOSP_N_1'] != 0), ['G201_HOSP_N_1', 'G201_HOSP_N_2', 'G201_HOSP_N_3', 'G201_HOSP_N_4']].value_counts()

G201_HOSP_N_1  G201_HOSP_N_2  G201_HOSP_N_3  G201_HOSP_N_4
1.0            0.0            88.0           88.0             43
                              0.0            88.0              7
                              1.0            88.0              7
                                             1.0               3
                              0.0            0.0               2
                                             1.0               2
                              1.0            0.0               2
                              2.0            88.0              1
2.0            0.0            0.0            0.0               1
Name: count, dtype: int64

For ID: 43600, it appears `HOSP_CODE_2` should be 102.  

In [50]:
df.loc[(df['G201_HOSP_N_2'] == 0) & (df['G201_HOSP_N_1'] != 0) & (df['G201_HOSP_N_3'] != 0), ['G201_HOSP_CODE_1', 'G201_HOSP_CODE_2', 'G201_HOSP_ICD9_1', 'G201_HOSP_ICD9_2',]]

,G201_HOSP_CODE_1,G201_HOSP_CODE_2,G201_HOSP_ICD9_1,G201_HOSP_ICD9_2
ID,,,,
11160.0,999.0,999.0,38290,99520
11800.0,999.0,999.0,78609,00900
12240.0,103.0,103.0,46610,38290
12830.0,103.0,999.0,59900,03810
13900.0,608.0,608.0,S6400,S3998
13950.0,102.0,102.0,11200,11230
14460.0,608.0,608.0,S0959,S0944
15330.0,103.0,103.0,55310,S6250
16310.0,631.0,631.0,78050,78330


Verify that when `HOSP_N_?` == 0, the corresponding `HOSP_CODE` and `HOSP_ICD9` values are not Missing or N/A, and that `HOSP_CODE` is equal to the previous `HOSP_CODE` value.  
TODO: continue fleshing out these tests.

In [51]:
df.loc[(df['G201_HOSP_N_1'] != 0) & (df['G201_HOSP_N_2'] == 0), ['G201_HOSP_CODE_1', 'G201_HOSP_CODE_2', 'G201_HOSP_ICD9_1', 'G201_HOSP_ICD9_2',]]

,G201_HOSP_CODE_1,G201_HOSP_CODE_2,G201_HOSP_ICD9_1,G201_HOSP_ICD9_2
ID,,,,
10290.0,999.0,999.0,S2161,S2830
11160.0,999.0,999.0,38290,99520
11800.0,999.0,999.0,78609,00900
12240.0,103.0,103.0,46610,38290
12830.0,103.0,999.0,59900,03810
...,...,...,...,...
50370.0,999.0,999.0,51980,69190
50920.0,102.0,102.0,34990,S8914
51110.0,999.0,999.0,46610,46440


### Update metadata

In [52]:
wb["Base Variable"] = (
    wb["Harmonised Variable Name"]
    .str
    .replace(r"N_([2-9]|1[0-2])", "N", regex=True)
    .replace(r"ICD9_\d+", "ICD9", regex=True)
    .replace(r"CODE_\d+", "CODE", regex=True)
)
m1 = wb[["vn_new", "Base Variable"]].rename(columns={"vn_new": "Variable"})
m2 = pd.read_excel(WORKBOOK, sheet_name="Harmonised")

updated_metadata = m1.merge(m2, on="Base Variable")
updated_metadata = format_metadata_pyspssio(updated_metadata)
meta[updated_metadata.columns] = updated_metadata

## Save Interim Data

In [53]:
write_sav(OUTPUT, df, meta)

## Testing and Validating Changes

In [54]:
df_old, meta_old = read_sav(INPUT, index="ID")
df_new, meta_new = read_sav(OUTPUT, index="ID")

### Data

Specify the precise changes expected from the updated dataset in terms of recoding.

In [55]:
# Changes should capture each specific changes; column (variable name), the IDs that are affected, and the corresponding change
CHANGES = [
    {
        "columns": ["G201_ILL_N_1"],
        "idx": [10080],
        "value": 1 
    },
    {
        "columns": ["G201_ILL_N_2"],
        "idx": [10080],
        "value": 0 
    },
    {
        "columns": ["G201_ILL_N_3", "G201_ILL_N_4"],
        "idx": [11642],
        "value": 88 
    },
    {
        "columns": ["G201_ILL_ICD9_3"],
        "idx": [11642],
        "value": "88888"
    },
    {
        "columns": ["G201_ILL_ICD9_4"],
        "idx": [11642],
        "value": "88888"
    },
    {
        "columns": ["G201_ILL_N_4"],
        "idx": [23180],
        "value": 88
    },
    {
        "columns": ["G201_ILL_ICD9_4"],
        "idx": [23180],
        "value": "88888"
    },
    {
        "columns": ['G201_ILL_ICD9_2', 'G201_ILL_ICD9_3', 'G201_ILL_ICD9_4', 'G201_ILL_ICD9_5', 'G201_ILL_ICD9_6', 'G201_ILL_ICD9_7', 'G201_ILL_ICD9_8', 'G201_ILL_ICD9_9', 'G201_ILL_ICD9_10', 'G201_ILL_ICD9_11', 'G201_ILL_ICD9_12'],
        "idx": [19270, 17890],
        "value": "88888"
    },
    {
        "columns": ['G201_ILL_N_2', 'G201_ILL_N_3', 'G201_ILL_N_4', 'G201_ILL_N_5', 'G201_ILL_N_6', 'G201_ILL_N_7', 'G201_ILL_N_8', 'G201_ILL_N_9', 'G201_ILL_N_10', 'G201_ILL_N_11', 'G201_ILL_N_12'],
        "idx": [19270, 17890],
        "value": 88
    },
    {
        "columns": ['G201_ILL_ICD9_4', 'G201_ILL_ICD9_5', 'G201_ILL_ICD9_6', 'G201_ILL_ICD9_7', 'G201_ILL_ICD9_8', 'G201_ILL_ICD9_9', 'G201_ILL_ICD9_10', 'G201_ILL_ICD9_11', 'G201_ILL_ICD9_12'],
        "idx": [20370],
        "value": "88888"
    },
    {
        "columns": ['G201_ILL_N_4', 'G201_ILL_N_5', 'G201_ILL_N_6', 'G201_ILL_N_7', 'G201_ILL_N_8', 'G201_ILL_N_9', 'G201_ILL_N_10', 'G201_ILL_N_11', 'G201_ILL_N_12'],
        "idx": [20370],
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_1"],
        "idx": [42320],
        "value": 999
    },
    {
        "columns": ["G201_HOSP_N_1"],
        "idx": idx_HOF1,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_2"],
        "idx": idx_HOF2,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_3"],
        "idx": idx_HOF3,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_4"],
        "idx": idx_HOF4,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_5"],
        "idx": idx_HOF5,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_N_6"],
        "idx": idx_HOF6,
        "value": 88
    },
    {
        "columns": ["G201_INJ_N_1"],
        "idx": idx_INF1,
        "value": 88
    },
    {
        "columns": ["G201_INJ_N_2"],
        "idx": idx_INF2,
        "value": 88
    },
    {
        "columns": ["G201_INJ_N_3"],
        "idx": idx_INF3,
        "value": 88
    },
    {
        "columns": ["G201_INJ_N_4"],
        "idx": idx_INF4,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_1"],
        "idx": idx_HOH1_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_1"],
        "idx": idx_HOH1_MISS,
        "value": 999
    },
    {
        "columns": ["G201_HOSP_CODE_2"],
        "idx": idx_HOH2_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_2"],
        "idx": idx_HOH2_MISS,
        "value": 999
    },
    {
        "columns": ["G201_HOSP_CODE_3"],
        "idx": idx_HOH3_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_3"],
        "idx": idx_HOH3_MISS,
        "value": 999
    },
    {
        "columns": ["G201_HOSP_CODE_4"],
        "idx": idx_HOH4_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_4"],
        "idx": idx_HOH4_MISS,
        "value": 999
    },
    {
        "columns": ["G201_HOSP_CODE_5"],
        "idx": idx_HOH5_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_5"],
        "idx": idx_HOH5_MISS,
        "value": 999
    },
    {
        "columns": ["G201_HOSP_CODE_6"],
        "idx": idx_HOH6_NA,
        "value": 88
    },
    {
        "columns": ["G201_HOSP_CODE_6"],
        "idx": idx_HOH6_MISS,
        "value": 999
    },
]

In [56]:
run_full_test_suite(df_new, df_old, CREATE, RENAME, DELETE, CHANGES)

### Metadata

Verify metadata for all unchanged variables (those not harmonised, and not intentionally removed), remains the same.

In [57]:
changed_variables = wb["vn_old"].dropna().to_list()
unchanged_variables = list(set(df_old.columns) - set(changed_variables))
unchanged_variables = [col for col in unchanged_variables if col not in DELETE]

In [58]:
pd.testing.assert_frame_equal(meta_new[unchanged_variables], meta_old[unchanged_variables])

Verify all harmonised variables (and those newly created) contain valid metadata.

In [59]:
harmonised_variables = wb["vn_new"].dropna().to_list()

In [60]:
meta_new[harmonised_variables]

,G201_ILL_N_1,G201_ILL_N_2,G201_ILL_N_3,G201_ILL_N_4,G201_ILL_N_5,G201_ILL_N_6,G201_ILL_N_7,G201_ILL_N_8,G201_ILL_N_9,G201_ILL_N_10,...,G201_HOSP_ICD9_6,G201_HOSP_N_1,G201_HOSP_N_2,G201_HOSP_N_3,G201_HOSP_N_4,G201_HOSP_N_5,G201_HOSP_N_6,G201_ILL,G201_INJ,G201_HOSP
Label,"Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...","Number of visits to hospital, doctor, or clini...",...,ICD-9 code for hospital admission,Number of visits to hospital,Number of visits to hospital,Number of visits to hospital,Number of visits to hospital,Number of visits to hospital,Number of visits to hospital,"Illness requiring a visit to hospital, doctor,...","Injury requiring a visit to hospital, doctor, ...",Admitted to hospital since last follow-up
Field Type,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,...,A5,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0,F3.0
Field Width,3,3,3,3,3,3,3,3,3,3,...,7,3,3,3,3,3,3,3,3,3
Decimals,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Variable Type,scale,scale,scale,scale,scale,scale,scale,scale,scale,scale,...,nominal,scale,scale,scale,scale,scale,scale,nominal,nominal,nominal
Field Values,"{-99.0: 'Missing', -88.0: 'N/A'}","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...",...,NaN,"{-99.0: 'Missing', -88.0: 'N/A'}","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', -88.0: 'N/A', 0.0: 'Same vi...","{-99.0: 'Missing', 0.0: 'No', 1.0: 'Yes'}","{-99.0: 'Missing', 0.0: 'No', 1.0: 'Yes'}","{-99.0: 'Missing', 0.0: 'No', 1.0: 'Yes'}"


## Recoding Missing and N/A values

In [61]:
# Outline all variables to be changed, what the original Missing and N/A values were, and what to change them to

ILL_N = ['G201_ILL_N_1', 'G201_ILL_N_2', 'G201_ILL_N_3', 'G201_ILL_N_4', 'G201_ILL_N_5', 'G201_ILL_N_6', 'G201_ILL_N_7', 'G201_ILL_N_8', 'G201_ILL_N_9', 'G201_ILL_N_10', 'G201_ILL_N_11', 'G201_ILL_N_12']
INJ_N = ['G201_INJ_N_1', 'G201_INJ_N_2', 'G201_INJ_N_3', 'G201_INJ_N_4']
HOSP_N = ['G201_HOSP_N_1', 'G201_HOSP_N_2', 'G201_HOSP_N_3', 'G201_HOSP_N_4', 'G201_HOSP_N_5', 'G201_HOSP_N_6']
HOSP_CODE = ['G201_HOSP_CODE_1', 'G201_HOSP_CODE_2', 'G201_HOSP_CODE_3', 'G201_HOSP_CODE_4', 'G201_HOSP_CODE_5', 'G201_HOSP_CODE_6']

ILL_ICD9 = ['G201_ILL_ICD9_1', 'G201_ILL_ICD9_2', 'G201_ILL_ICD9_3', 'G201_ILL_ICD9_4', 'G201_ILL_ICD9_5', 'G201_ILL_ICD9_6', 'G201_ILL_ICD9_7', 'G201_ILL_ICD9_8', 'G201_ILL_ICD9_9', 'G201_ILL_ICD9_10', 'G201_ILL_ICD9_11', 'G201_ILL_ICD9_12']
INJ_ICD9 = ['G201_INJ_ICD9_1', 'G201_INJ_ICD9_2', 'G201_INJ_ICD9_3', 'G201_INJ_ICD9_4']
HOSP_ICD9 = ['G201_HOSP_ICD9_1', 'G201_HOSP_ICD9_2', 'G201_HOSP_ICD9_3', 'G201_HOSP_ICD9_4', 'G201_HOSP_ICD9_5', 'G201_HOSP_ICD9_6']

GROUP_A = [*ILL_N, *INJ_N, *HOSP_N, *HOSP_CODE]
GROUP_B = [*ILL_ICD9, *INJ_ICD9, *HOSP_ICD9]

In [62]:
df_new[GROUP_A] = df_new[GROUP_A].replace({88: -88, 999: -99})
df_new[GROUP_B] = df_new[GROUP_B].replace({"88888": "N/A", "99999": "Missing"})

## Save updated dataset

In [63]:
write_sav(FINAL, df_new, meta_new)